### User Input

In [1]:
# MODEL PARAMETERS
model_settings ={
    'layers': 7,
    'dropout': 0.1,
    'padding': 'same',
    'n_models_low': 16,
    'n_models_high': 16,
    'model_skip_rate': 16,
    'trial': 4
}

### Path Setup

In [2]:
layer_count = str(model_settings['layers'])
count_low = str(model_settings['n_models_low'])
count_high = model_settings['n_models_high']
count_high_end = str(model_settings['model_skip_rate']*model_settings['n_models_high'])
trial_count = str(model_settings['trial'])

mfccs_json_path = "../mfccs_cnn_humpbackwhale_walrus_bowheadwhale_fin_finbackwhale_killerwhale_emptyocean.json"

csv_file_name = f'../model-stats/layers-{layer_count}_filters-1-{count_low}-600-{count_high_end}_n-trail{trial_count}.csv'

plot_file_name = f'../images/layers-{layer_count}_filters-1-{count_low}-600-{count_high_end}_n-trail{trial_count}.png'

### Imports

In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np
import warnings

### Load MFCC Dataset

In [4]:
from cnn_comparison_lib import load_cnn_json

X, y, L = load_cnn_json(mfccs_json_path)
print(f"mapping the marine mammals: {L}")

mapping the marine mammals: ['BowheadWhale' 'HumpbackWhale' 'KillerWhale' 'Walrus' 'Fin_FinbackWhale'
 'EmptyOcean']


In [5]:
# Check class counts and that labels are numerical order
np.unique(y, return_counts=True)

(array([1, 2, 3, 4, 5, 6]), array([1000, 1000, 1000, 1000, 1000, 1000]))

### Create Training, Validation, and Test Sets

In [6]:
# create train, validation and test sets
from cnn_comparison_lib import prepare_datasets

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X, y, 0.6, 0.5) # test size, vailidation size
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

### Build the CNN Models

In [7]:
from cnn_comparison_lib import create_layered_cnn

model_list = []
for i in range(model_settings['n_models_low']):
    output_shape = i + 1
    model = create_layered_cnn(model_settings['layers'], input_shape, output_shape, model_settings)
    model_list.append(model)

In [8]:
from cnn_comparison_lib import create_layered_cnn

for i in range(model_settings['n_models_high']):
    output_shape = 600 + (i*10)
    model = create_layered_cnn(model_settings['layers'], input_shape, output_shape, model_settings)
    model_list.append(model)

### Get Trainable Parameter Count

In [9]:
from cnn_comparison_lib import count_trainable_parameters

parameter_count = []
for model in model_list:
    count = count_trainable_parameters(model)
    parameter_count.append(count)

print(parameter_count)

[108763, 217115, 325575, 434143, 542819, 651603, 760495, 869495, 978603, 1087819, 1197143, 1306575, 1416115, 1525763, 1635519, 1745383, 84354519, 86089819, 87835919, 89592819, 91360519, 93139019, 94928319, 96728419, 98539319, 100361019, 102193519, 104036819, 105890919, 107755819, 109631519, 111518019]


### Train the Models

In [10]:
from cnn_comparison_lib import plot_history

model_history = []
for model in model_list:
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=16, epochs=5)
    model_history.append(history)

Epoch 1/5


2023-03-06 12:12:23.368914: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


75/75 [==============================] - 4s 27ms/step - loss: 1.5870 - accuracy: 0.3875 - val_loss: 2.2248 - val_accuracy: 0.1575
Epoch 2/5
75/75 [==============================] - 2s 24ms/step - loss: 1.2752 - accuracy: 0.5150 - val_loss: 2.7207 - val_accuracy: 0.1575
Epoch 3/5
75/75 [==============================] - 2s 26ms/step - loss: 1.1799 - accuracy: 0.5583 - val_loss: 2.9537 - val_accuracy: 0.1575
Epoch 4/5
75/75 [==============================] - 2s 25ms/step - loss: 1.0830 - accuracy: 0.5808 - val_loss: 2.7088 - val_accuracy: 0.1575
Epoch 5/5
75/75 [==============================] - 2s 25ms/step - loss: 1.0217 - accuracy: 0.6050 - val_loss: 2.1100 - val_accuracy: 0.2008
Epoch 1/5
75/75 [==============================] - 4s 32ms/step - loss: 1.4797 - accuracy: 0.4775 - val_loss: 2.1478 - val_accuracy: 0.1608
Epoch 2/5
75/75 [==============================] - 2s 30ms/step - loss: 0.9523 - accuracy: 0.6658 - val_loss: 2.5682 - val_accuracy: 0.1608
Epoch 3/5
75/75 [=============

### Evaluate the Models

In [ ]:
# evaluate the cnn on the test set
model_accuracy = []
for model in model_list:
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    model_accuracy.append(test_accuracy)

### Plot the Models

In [ ]:
x = parameter_count
y = model_accuracy
plt.plot(x, y, 'o', color='black')
plt.xlabel("Number of Parameters")
plt.ylabel("Model Accuracy")
plt.savefig(plot_file_name)

In [ ]:
print(parameter_count)
print(model_accuracy)

### Save the Models

In [ ]:
"""saved_models_path = "../saved_model/layers" + str(model_settings['layers']) + '/trial' + str(model_settings['trial']) + "/paramcount"
for n_params, model in zip(parameter_count, model_list):
    model_name = saved_models_path + str(n_params) + '.h5'
    model.save(model_name)
"""

### Save the Model Parameters and Accuracy Values

In [ ]:
"""
import csv
from itertools import zip_longest

list1 = parameter_count
list2 = model_accuracy
d = [list1, list2]
export_data = zip_longest(*d, fillvalue = '')
with open(csv_file_name, 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("Parameter Count", "Baseline Accuracy"))
      wr.writerows(export_data)
myfile.close()
"""